In [1]:
# Install required packages
!pip install opencv-python numpy pyaudio

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import threading
import time
import pyaudio
import math
import wave

# Video parameters
WIDTH = 800
HEIGHT = 400
FPS = 30
DURATION = 10  # seconds
TOTAL_FRAMES = FPS * DURATION

# Audio parameters
SAMPLE_RATE = 44100
FREQUENCY = 440  # Hz (A4 note)
CHUNK_SIZE = 1024

# Dot parameters
DOT_RADIUS = 20
DOT_COLOR = (0, 255, 0)  # Green

print("Configuration loaded successfully!")

Configuration loaded successfully!


In [3]:
class AudioGenerator:
    def __init__(self):
        self.audio = pyaudio.PyAudio()
        self.stream = None
        self.running = False
        self.dot_position = 0.5  # 0.0 = left, 1.0 = right
        
    def start_audio(self):
        self.stream = self.audio.open(
            format=pyaudio.paFloat32,
            channels=2,  # Stereo
            rate=SAMPLE_RATE,
            output=True,
            frames_per_buffer=CHUNK_SIZE
        )
        self.running = True
        
    def generate_audio_chunk(self):
        frames = np.arange(CHUNK_SIZE)
        wave = np.sin(2 * np.pi * FREQUENCY * frames / SAMPLE_RATE)
        
        # Calculate stereo panning
        left_volume = 1.0 - self.dot_position
        right_volume = self.dot_position
        
        # Create stereo audio
        left_channel = wave * left_volume * 0.3  # Reduce volume
        right_channel = wave * right_volume * 0.3
        
        # Interleave stereo channels
        stereo_audio = np.column_stack((left_channel, right_channel))
        return stereo_audio.astype(np.float32)
        
    def update_position(self, position):
        self.dot_position = position
        
    def play_audio(self):
        while self.running:
            audio_chunk = self.generate_audio_chunk()
            self.stream.write(audio_chunk.tobytes())
            
    def stop_audio(self):
        self.running = False
        if self.stream:
            self.stream.stop_stream()
            self.stream.close()
        self.audio.terminate()

print("AudioGenerator class created successfully!")

AudioGenerator class created successfully!


In [4]:
def create_video_with_audio():
    """Generate video file with moving dot and synchronized stereo audio"""
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter('moving_dot.mp4', fourcc, FPS, (WIDTH, HEIGHT))
    
    # Initialize audio
    audio_gen = AudioGenerator()
    audio_gen.start_audio()
    
    # Start audio thread
    audio_thread = threading.Thread(target=audio_gen.play_audio)
    audio_thread.daemon = True
    audio_thread.start()
    
    print("Generating video with stereo audio...")
    
    for frame_num in range(TOTAL_FRAMES):
        # Calculate dot position (oscillating from left to right)
        t = frame_num / TOTAL_FRAMES
        dot_x = int((math.sin(2 * math.pi * t * 2) + 1) / 2 * (WIDTH - 2 * DOT_RADIUS) + DOT_RADIUS)
        dot_y = HEIGHT // 2
        
        # Update audio panning based on dot position
        audio_position = (dot_x - DOT_RADIUS) / (WIDTH - 2 * DOT_RADIUS)
        audio_gen.update_position(audio_position)
        
        # Create frame
        frame = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
        cv2.circle(frame, (dot_x, dot_y), DOT_RADIUS, DOT_COLOR, -1)
        
        # Add position indicator
        cv2.putText(frame, f"Position: {audio_position:.2f}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # Write frame
        video_writer.write(frame)
        
        # Display frame (optional)
        cv2.imshow('Moving Dot', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        # Control frame rate
        time.sleep(1/FPS)
    
    # Cleanup
    audio_gen.stop_audio()
    video_writer.release()
    cv2.destroyAllWindows()
    
    print("Video saved as 'moving_dot.mp4'")

print("Video generation function created!")

Video generation function created!


In [5]:
def play_realtime_demo():
    """Play the dot movement with real-time audio (no video recording)"""
    audio_gen = AudioGenerator()
    audio_gen.start_audio()
    
    # Start audio thread
    audio_thread = threading.Thread(target=audio_gen.play_audio)
    audio_thread.daemon = True
    audio_thread.start()
    
    print("Playing real-time demo. Press 'q' to quit.")
    
    start_time = time.time()
    
    while True:
        current_time = time.time() - start_time
        
        # Calculate dot position
        t = current_time / DURATION
        dot_x = int((math.sin(2 * math.pi * t * 2) + 1) / 2 * (WIDTH - 2 * DOT_RADIUS) + DOT_RADIUS)
        dot_y = HEIGHT // 2
        
        # Update audio panning
        audio_position = (dot_x - DOT_RADIUS) / (WIDTH - 2 * DOT_RADIUS)
        audio_gen.update_position(audio_position)
        
        # Create frame
        frame = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
        cv2.circle(frame, (dot_x, dot_y), DOT_RADIUS, DOT_COLOR, -1)
        
        # Add indicators
        cv2.putText(frame, f"Audio Position: {audio_position:.2f}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, "Press 'q' to quit", (10, HEIGHT - 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Display frame
        cv2.imshow('Moving Dot with Stereo Audio', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        time.sleep(1/FPS)
    
    # Cleanup
    audio_gen.stop_audio()
    cv2.destroyAllWindows()

print("Real-time demo function created!")

Real-time demo function created!


In [6]:
# Choose what to run:
# Uncomment one of the following lines:

# For real-time demo with audio (recommended to test first):
play_realtime_demo()

# For generating a video file:
# create_video_with_audio()

Playing real-time demo. Press 'q' to quit.
